In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import warnings
warnings.filterwarnings('ignore')

import os
relative_path = "../__Fitting_Drivers/"
absolute_path = os.path.abspath(relative_path)

import sys
import time
import json
from tqdm import tqdm

sys.path.append(absolute_path)

In [2]:
from sklearn.neural_network import MLPRegressor
from Scaler_driver import DataScaler as DS
# Import drivers
import Surrogate_driver_sklearn as SD
import PSO_driver_withscaler as PSO
import Simulation_driver as SM


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models
from keras.saving import register_keras_serializable

# --- Custom Monte Carlo Dropout Layer ---
@register_keras_serializable()
class MCDropout(layers.Dropout):
    def call(self, inputs, training=None):
        return super().call(inputs, training=True)

# --- Build BNN ---
def build_bnn_model(input_dim, output_dim, dropout_rate=0.1):
    model = models.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(50, activation='relu'),
        MCDropout(dropout_rate),
        layers.Dense(output_dim)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# --- Predict with Uncertainty ---
def predict_with_uncertainty(f_model, x_tensor, n_iter=50):
    preds = np.stack([f_model(x_tensor, training=True).numpy() for _ in range(n_iter)], axis=0)
    mean_preds = preds.mean(axis=0)
    std_preds = preds.std(axis=0)
    return mean_preds, std_preds

In [3]:
process_data = pd.read_excel('../01_DataCollecting/Sim_result/Heat_exchanger_process_data_result_V5-short.xlsx', index_col=0)
process_data.reset_index(drop=True, inplace=True)

training_data = pd.read_csv('../01_DataCollecting/Sim_result/Surrogate_dataset_LHS_200.csv', index_col=0, encoding='ISO-8859-1')
training_data.reset_index(drop=True, inplace=True)

#direct_calibration = pd.read_excel('../03_Calibration_offline/Results/Direct_sim/Direct_Calibration_result_offline.xlsx', index_col=0)

In [4]:
from CalibrationWorkflow_driver import CalibrationWorkflow

In [5]:
y_fitt = {'PROD1E1IN': {'Weight': 1, 'SDEV':1},
  'PROD1OUT': {'Weight': 1, 'SDEV':1},
  'FEED1E1OUT': {'Weight': 1, 'SDEV':1},
  'FEED1OUT': {'Weight': 1, 'SDEV':1}}

absminfouling = 0.00025
absmaxfouling = 0.002936
# minimum hard limit az optimizernek , 'E1FOULING': (0.00025, 0.002936)
fitparamlimit = {'E1FOULING': {'min':absminfouling, 'max':absmaxfouling}, 'E2FOULING': {'min':absminfouling, 'max':absmaxfouling}}

In [6]:
# Simulation Cols mapping
cols_mapping = {'InputParams': {'FEED1MASSFLOW': 'C3', 'FEED1TEMP': 'C4','PROD1MASSFLOW': 'C5','PROD1TEMP': 'C6',
                          'E1FOULING': 'C11', 'E2FOULING': 'C15'},
               'OutputParams': {'PROD1E1IN': 'C3', 'PROD1OUT': 'C4', 'FEED1E1OUT': 'C7', 'FEED1OUT': 'C8'},
               'Outputuoms': {'PROD1E1IN': 'C', 'PROD1OUT': 'C', 'FEED1E1OUT': 'C', 'FEED1OUT': 'C'}
               }

        # always-one: If direct calibration data exists and is accurate, use that; else use validation data if accurate.
        # always-both: Use both validation and direct calibration data if available and accurate.
        # conditional: Only use direct calibration data if it exists and is accurate.
workflow_settings = {'calibration_type': 'surrogate', # direct, surrogate, online
                     'run_direct_for_not_accurate': False,
                     'stop_workingpoint' : None, # None 
                     'workflow_random_seed': None, # None
                     'debug_workflow': False,
                     'sim_validation_treshold' : 0.1, # None
                     'base_calibration_error_factor': 0.1,
                     'point_selection_type': 'always-one',  # always-one, always-both, conditional
                     'logging_level': 'INFO',  # DEBUG, INFO, WARNING, ERROR, CRITICAL
                     }
calibration_settings = {'y_fitt': y_fitt,
                        'fitparamlimit': fitparamlimit,
                        'particle_num': 20,
                        'optim_iterations': 100,
                        'c1' : 0.1,
                        'c2' : 0.4,
                        'w': 0.7,
                        'stopping_treshold' : 0.0001,
                        'stopping_obj' : 0.1,
                        'optim_debug' : False,
                        }

simulation_model_setting = { 'hy_filename' : '../00_Modell_database/Simulation/SampleModel_V2.hsc',
                            'cols_mapping' : cols_mapping, 
                            'resultindict' : True, 
                            }

surrogate_model_settings = {'load_path' : '../00_Modell_database/AdaptiveModel/sklearnann.joblib'}

online_learning_settings = {'batch_size': 1, 'replay_ratio': 0.01, 'retrain_scaler': False}


In [7]:
# Initialize the calibration workflow with the provided settings and models
workflow = CalibrationWorkflow(process_data, workflow_settings = workflow_settings, calibration_settings=calibration_settings)
workflow.set_simulation_model(simulation_model_setting = simulation_model_setting)
workflow.set_surrogate_model(surrogate_model_settings = surrogate_model_settings)
workflow.init_models()
workflow.init_online_learning(training_data, **online_learning_settings)

[INFO] 20:56:21 - CalibrationWorkflow: Simulation model is initialized, and model loaded from: SampleModel_V2.hsc
[INFO] 20:56:21 - CalibrationWorkflow: Surrogate model is initialized from: ../00_Modell_database/AdaptiveModel/sklearnann.joblib
[INFO] 20:56:21 - CalibrationWorkflow: Online learning initialized with surrogate model and settings : {'batch_size': 1, 'replay_ratio': 0.01, 'retrain_scaler': False}


In [8]:
workflow.run_calibration()

[INFO] 20:56:22 - CalibrationWorkflow: 
[INFO] 20:56:22 - CalibrationWorkflow: STARTING CALIBRATION WORKFLOW.
[INFO] 20:56:22 - CalibrationWorkflow: ============================================================

[INFO] 20:56:22 - CalibrationWorkflow: ----------------------------------------
[INFO] 20:56:22 - CalibrationWorkflow: [PRIMARY-CALIBRATION] Processing sample 0 of 529.
[INFO] 20:56:22 - CalibrationWorkflow: ----------------------------------------
[INFO] 20:56:30 - CalibrationWorkflow: Obj Treshold reached with 10 iteration, best cost: 1.780912014010695e-06, values: {'FEED1MASSFLOW': 18035.53430133957, 'FEED1TEMP': 25.25656349837262, 'PROD1MASSFLOW': 45159.7172098537, 'PROD1TEMP': 116.5605783388438, 'E1FOULING': 0.00025, 'E2FOULING': 0.0004939975142064824}
[INFO] 20:56:31 - CalibrationWorkflow: Working point is NOT accurate enough with cost: 0.263091 > threshold: 0.1
[INFO] 20:56:31 - CalibrationWorkflow: ----------------------------------------
[INFO] 20:56:31 - CalibrationWor

In [ ]:
workflow.save_results_df('./Results/Surrogate_base/02_Calibration_static_BNN.xlsx')

[INFO] 22:54:31 - CalibrationWorkflow: Calibration results saved to ./Results/Surrogate_base/Calibration_adaptive_surrogate_no_direct.xlsx
